# eRO-ExTra database setup

This notebook creates a SQL database **eroextra.db** containing the full catalog of 304 eRO-ExTra sources.

### The following tables are created:

- **"eroextra_master_erosita"** - the master catalog of 304 X-ray sources with eROSITA with their key X-ray properties
- **"optical_follow_up"** - a table of optical follow-up observations conducted this study, linked to corresponding X-ray sources

In [1]:
import sqlite3
from astropy.io import fits
import ast

In [2]:
#load the master ero-extra catalog in fits format
eroextra_catalog_fits = 'data/eroextra_final.fits'
with fits.open(eroextra_catalog_fits) as hdul:
    eroextra_data = hdul[1].data

In [3]:
#load the text file with optical follow-up data
with open("data/optical_data.txt", "r") as f:
    content = f.read()
optical_data = ast.literal_eval(content)

In [4]:
conn = sqlite3.connect("eroextra.db")
cursor = conn.cursor()

In [5]:
# Drop tables if re-running
cursor.execute("DROP TABLE IF EXISTS optical_follow_up;")
cursor.execute("DROP TABLE IF EXISTS eroextra_master_erosita;")

In [6]:
# Create the eroextra_master_erosita table
cursor.execute("""
CREATE TABLE eroextra_master_erosita (
    ero_name TEXT PRIMARY KEY,
    ero_ra REAL,
    ero_dec REAL,
    ero_poserr REAL,
    ero_flux_erass1 REAL,
    ero_flux_erass2 REAL,
    ero_flux_erass3 REAL,
    ero_flux_erass4 REAL,
    ero_flux_erass5 REAL,
    ero_flux_err_erass1 REAL,
    ero_flux_err_erass2 REAL,
    ero_flux_err_erass3 REAL,
    ero_flux_err_erass4 REAL,
    ero_flux_err_erass5 REAL,
    amplitude REAL,
    significance REAL,
    ero_lcclass REAL,
    ero_date TEXT,
    nh REAL,
    ero_gamma REAL,
    ero_gamma_err REAL,
    ero_cstat_dof TEXT,
    ero_flux_mod REAL,
    ero_flux_mod_err REAL
);
""")

In [8]:
insert_sql = """
INSERT OR REPLACE INTO eroextra_master_erosita (
    ero_name, ero_ra, ero_dec, ero_poserr,
    ero_flux_erass1, ero_flux_erass2, ero_flux_erass3, ero_flux_erass4, ero_flux_erass5,
    ero_flux_err_erass1, ero_flux_err_erass2, ero_flux_err_erass3, ero_flux_err_erass4, ero_flux_err_erass5,
    amplitude, significance, ero_lcclass, ero_date, nh,
    ero_gamma, ero_gamma_err, ero_cstat_dof, ero_flux_mod, ero_flux_mod_err
) VALUES (
    ?, ?, ?, ?,
    ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?
);
"""

for row in eroextra_data:
    values = (
        row['ero_name'],
        float(row['ero_ra']),
        float(row['ero_dec']),
        float(row['ero_poserr']),
        float(row['ero_flux_erass1']),
        float(row['ero_flux_erass2']),
        float(row['ero_flux_erass3']),
        float(row['ero_flux_erass4']),
        float(row['ero_flux_erass5']),
        float(row['ero_flux_err_erass1']),
        float(row['ero_flux_err_erass2']),
        float(row['ero_flux_err_erass3']),
        float(row['ero_flux_err_erass4']),
        float(row['ero_flux_err_erass5']),
        float(row['amplitude']),
        float(row['significance']),
        float(row['ero_lcclass']),
        row['ero_date'],  # text, no conversion needed
        float(row['nh']),
        float(row['ero_gamma']),
        float(row['ero_gamma_err']),
        row['ero_cstat_dof'],  # text, no conversion
        float(row['ero_flux_mod']),
        float(row['ero_flux_mod_err'])
    )
    cursor.execute(insert_sql, values)

conn.commit()

In [9]:
cursor.execute("SELECT * FROM eroextra_master_erosita LIMIT 5;")
rows = cursor.fetchall()

for row in rows:
    print(row)

('1eRASS J000258.3-401950', 0.7411180461535105, -40.33086651370921, 0.9081184688516774, 1.7011042639618335e-13, 7.950998258225705e-13, 5.51243946322888e-13, 1.1131085202870095e-12, None, 4.404208117994579e-14, 8.59647097430083e-14, 7.625111208674901e-14, 1.0677544867157204e-13, None, 4.674021708527148, 6.470536846692182, 4.0, '(58994.6,59181.8,59363.1,59548.7)', 1.2299999965027815e+20, 1.9002655901512076, 0.18541015403361488, '1158.15/1549', 8.69272297182996e-13, 9.216420358842665e-14)
('1eRASS J001053.3-334021', 2.7223018808188932, -33.67247573213791, 1.6461938656361548, 6.005892940093294e-13, 1.20545297376852e-13, 9.911309376301955e-14, 6.652233560007492e-14, None, 8.301912217574536e-14, 4.609612930077389e-14, -1.0, 2.7215575171095352e-14, None, 4.982270624226433, 5.055328144627522, 1.0, '(58998.5,59185.5,59367.0,59553.6)', 1.3200000370092697e+20, 1.9852319812169192, 0.2349214548569039, '1015.87/1549', 6.402407214959363e-13, 8.72574632202884e-14)
('eRASSU J001308.6-462524', 3.2856429

In [10]:
# Create the optical_follow_up table
cursor.execute("""
CREATE TABLE optical_follow_up (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ero_name TEXT,
    ra REAL,
    dec REAL,
    redshift REAL,
    date TEXT,
    telescope TEXT,
    instrument TEXT,
    FOREIGN KEY (ero_name) REFERENCES eroextra_master_erosita(ero_name)
);
""")

insert_sql = """
INSERT INTO optical_follow_up 
    (ero_name, ra, dec, redshift, date, telescope, instrument) 
VALUES (?, ?, ?, ?, ?, ?, ?);
"""

# Insert all rows in a batch
cursor.executemany(insert_sql, optical_data)

In [12]:
cursor.execute("SELECT * FROM optical_follow_up LIMIT 10;")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'eRASSU J001308.6-462524', 3.2844, -46.422, 0.198, '2022-07-03', 'La Silla/NTT', 'EFOSC2')
(2, '1eRASS J004058.3-683816', 10.2434, -68.6388, 0.151, '2023-12-25', 'La Silla/NTT', 'EFOSC2')
(3, 'eRASSU J011430.8-593654', 18.6292, -59.6156, 0.156, '2020-12-12', 'Las Campanas/Baade', 'IMACS')
(4, '1eRASS J014133.4-443413', 25.3891, -44.5706, 0.091, '2021-12-06', 'La Silla/NTT', 'EFOSC2')
(5, '1eRASS J015752.7-520716', 29.4716, -52.1208, 0.499, '2023-12-23', 'La Silla/NTT', 'EFOSC2')
(6, '1eRASS J022756.0-840730', 36.9827, -84.1246, 0.102, '2022-12-31', 'Siding Spring/LCO', 'FLOYDS')
(7, '1eRASS J024140.3-422435', 40.4171, -42.4102, 0.214, '2023-02-03', 'La Silla/NTT', 'EFOSC2')
(8, '1eRASS J024930.1-274958', 42.3771, -27.8327, 0.089, '2023-12-23', 'La Silla/NTT', 'EFOSC2')
(9, 'eRASSU J030334.1-544438', 45.8917, -54.7442, 0.091, '2020-10-14', 'Sutherland/SALT', 'RSS')
(10, 'eRASSU J034425.8-332719', 56.1084, -33.4553, 0.092, '2020-08-22', 'Siding Spring/ANU 2.3m', 'WiFeS')


In [15]:
#commit and close the connection
conn.commit()
conn.close()

In [16]:
conn = sqlite3.connect("eroextra.db")
cursor = conn.cursor()

# Show all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

conn.close()

[('sqlite_sequence',), ('eroextra_master_erosita',), ('optical_follow_up',)]
